CREATING FUNCTION FOR CITY TABLE

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse    # for decimal coordinates
from dotenv import load_dotenv
import os
load_dotenv()
schema = "city_workshop" # your schema
host = "145.0.0.2" # your host data base on MYSQL
user = "root"
password = os.getenv("MYSQL_PASSWORD") # replace with your password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def get_city_data(cities):

    city_data = []

    for city in cities:
        url= f"https://en.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')

        try:
            country = city_soup.find(class_='infobox-data').get_text()
            city_longitude = city_soup.find(class_='longitude').get_text()
            city_latitude = city_soup.find(class_='latitude').get_text()

        except AttributeError:
            country, latitude, longitude = None, None, None  # Handle missing values

        # extract data
        city_data.append({
            "city_name" : city,
            "Country" : country,
            "Latitude" : parse(city_latitude),
            "Longitude" : parse(city_longitude)
        })
    cities_df = pd.DataFrame(city_data)
    cities_df.to_sql('city', if_exists="append", con=connection_string , index=False)
    
    return "Data has been sent to the table", 200


CREATING FUNCTION FOR POPULATION TABLE

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime # to get today's date
load_dotenv()
schema = "city_workshop" # your schema
host = "145.0.0.2" # your host data base on MYSQL
user = "root"
password = os.getenv("MYSQL_PASSWORD") # replace with your password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def Get_Population_Data(cities):

    population_data = []
    # make a loop
    for city in cities:
        url = f"https://en.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        Pop_soup = BeautifulSoup(response.content, 'html.parser')

        # extract the data
        try:
            city_pop = Pop_soup.find(string='Population').find_next('td').get_text(strip=True)
            city_pop_clean = int(city_pop.replace(",", ""))
            today = datetime.today().strftime("%d.%m.%Y")

        except AttributeError:
            city_pop_clean = None, None, None  # Handle missing values

        # values for each city
        population_data.append({
            "city_name" : city,
            "Population": city_pop_clean,
            "Population_Timestamp":today
        })
    population = pd.DataFrame(population_data)
    population_df_raw = population.merge(pd.read_sql('city', con=connection_string), on ="city_name", how= "left") # merging city table to get city_id
    population_df =population_df_raw[["Population", "Population_Timestamp", "City_id"]] # select important rows for your population table
    population_df.to_sql('population', if_exists='append', con=connection_string, index=False) # sending your dataframe to MY sql database for storage

    return "Data has been sent to the table", 200

CREATING FUNCTION FOR WEATHER TABLE

In [ ]:
import requests
import pandas as pd
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()
schema = "city_workshop"
host = "145.0.0.2" # your host data base on MYSQL
user = "root"
password = os.getenv("MYSQL_PASSWORD") # replace with your password
port = 3306
key = os.getenv("WEATHER_API_KEY") # replace with your weather API key
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def get_city_weather_data(cities):

    weather_data = [] # empty list to store weather data
    
    cities_df = pd.read_sql("city", con=connection_string) # read city Id from sql
    for i, row in cities_df.iterrows():
         
        URL = f"https://api.openweathermap.org/data/2.5/forecast?lat={row['Latitude']}&lon={row['Longitude']}&appid={key}&units=metric"
        response = requests.get(URL)
        if response.status_code == 200:     # Only extract data if call was successful
            weather_json = response.json()
            for data in weather_json['list']:     
                weather_data.append({
                    "city_id": row['City_id'],
                    "city_name": row['city_name'],
                    "forecast_time": data.get('dt_txt'),
                    "timestamp_weather" : datetime.today().strftime("%Y-%m-%d %H:%M:%S"),
                    "outlook" : data.get('weather', [{}])[0].get('description', None),
                    "temperature": data.get('main', {}).get("temp", None),
                    "feels_Like": data.get('main', {}).get("feels_like", None),
                    "temp_Min": data.get('main', {}).get("temp_min", None),
                    "temp_Max": data.get('main', {}).get("temp_max", None), 
                    "pressure": data.get('main', {}).get("pressure", None), 
                    "humidity": data.get('main', {}).get("humidity", None), 
                    "wind_speed" : data.get('wind', {}).get('speed', None),
                    "rain" : data.get('rain', {}).get('3h', 0)
                })
        else:
            print(f"Error {response.status_code} at {row['id']}")
    weather_df = pd.DataFrame(weather_data)
    weather_df['forecast_time'] = pd.to_datetime(weather_df['forecast_time']) # change forecast_time to datetime 
    weather_df['timestamp_weather'] = pd.to_datetime(weather_df['timestamp_weather']) # change timestamp_weather to datetime 
    weather_df.to_sql('weather', if_exists="append", con=connection_string, index=False)
    
    return "Data has been sent to the table", 200

CREATING FUNCTION FOR CITIES AIRPORT TABLE

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env into environment

schema = "city_workshop" # name of your data base on MySQL
host = "145.0.0.2" # your host data base on MYSQL
user = "root"
password = os.getenv("MYSQL_PASSWORD")
port = 3306
api_key = os.getenv("AEROBOX_API_KEY")

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
def icao_airport_codes(cities_df):
    icao_codes =  []
    city_id = []
    airport_name = []

    cities_df = pd.read_sql("city", con=connection_string)

    for index, row in cities_df.iterrows():
        latitude = row['Latitude'] # be aware of column names in your SQL table
        longitude = row['Longitude'] # be aware of column names in your SQL table
        
        
        
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location"

        querystring = {"lat": latitude,
                       "lon":longitude,
                       "radiusKm":"50",
                       "limit":"10",
                       "withFlightInfoOnly":"true"}

        headers = {
	        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
            "X-RapidAPI-Key": api_key #"YOUR_API_KEY_HERE"
        }

        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            airport_json = response.json()
        else:
            return response.reason
        
        # to append the data
        for data in airport_json['items']:
            icao_codes.append(data['icao'])
            city_id.append(row['City_id'])
            airport_name.append(data['name'])
            
        
    cities_airport = pd.DataFrame({"city_id": city_id, "icao_codes": icao_codes, "airport_name" : airport_name })
    cities_airport.to_sql("cities_airport", if_exists='append', index=False, con=connection_string)
    return  "Data has been sent to the table", 200

CREATING FUNCTION FOR FLIGHT TABLE

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env into environment

schema = "city_workshop" # name of your data base on MySQL
host = "145.0.0.2" # your host data base on MYSQL
user = "root"
password = os.getenv("MYSQL_PASSWORD")
port = 3306
api_key = os.getenv("AEROBOX_API_KEY")

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def get_flight_data(icao_list):
        berlin_timezone = timezone('Europe/Berlin')
        today = datetime.now(berlin_timezone).date()
        tomorrow = (today + timedelta(days=1))

        flight_data =[]
        
        airport_df= pd.read_sql("cities_airport", con=connection_string)
        
        for i, row in airport_df.iterrows():
                # the api can only make 12 hour calls, therefore, two 12-hour calls make a full day
                city_id = row['city_id']
                icao_codes = row['icao_codes']
                                
                times = [["00:00", "11:59"], ["12:00", "23:59"]]
                
                for time in times:
                        
                        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao_codes}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
                        querystring = {"withLeg":"true",
                                "direction":"Both",
                                "withCancelled":"true",
                                "withCodeshared":"true",
                                "withCargo":"false",
                                "withPrivate":"false",
                                "withLocation":"false"}

                        headers = {
                                'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
                                'x-rapidapi-key': api_key
                        }
                        response = requests.get(url, headers=headers, params=querystring)
                        if response.status_code == 200:
                                flight_json = response.json()

                                retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

                                for item in flight_json.get("arrivals", []):  # Safely get 'arrivals'
                                        flight_data.append ({
                                        "city_id" : city_id,
                                        "arrival_airport_icao": icao_codes,
                                        "flight_number": item.get("number", None),
                                        "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                                        "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                                        "timestamp_flight": retrieval_time
                                                                               
                                        })
                                        
                        else:
                                print(f"Error fetching data for ICAO {icao_codes}: {response.status_code} - {response.text}")

        df = pd.DataFrame(flight_data)
                # Ensure proper datetime format
        
        if not df.empty:
                df["scheduled_arrival_time"] = df["scheduled_arrival_time"].str[:-6] # to remove unwanted format
                df["scheduled_arrival_time"] = pd.to_datetime(df["scheduled_arrival_time"]) # to change to datetime
                df["timestamp_flight"] = pd.to_datetime(df["timestamp_flight"]) # to change to datetime
                df = df.drop_duplicates()
                # Save to DB
                df.to_sql("flight", if_exists='append', index=False, con=connection_string)

        return "Data has been sent to the table", 200